In [ ]:
from lxml import etree, html as lhtml

In [ ]:
import requests
from lxml import html

url = 'https://automobili.ru/cars/catalog/'
response = requests.get(url)
tree = html.fromstring(response.content)

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
from operator import attrgetter, itemgetter

In [ ]:
brand_links = []
n = 0
for a_tag in soup.find_all('a', class_='car-brand-link'):
    brand_links.append(a_tag['href'])
    n+=1

print("Ссылки на бренды:", n, brand_links)

Ссылки на бренды: 52 ['/cars/catalog/aston_martin/', '/cars/catalog/audi/', '/cars/catalog/bentley/', '/cars/catalog/bmw/', '/cars/catalog/bugatti/', '/cars/catalog/cadillac/', '/cars/catalog/changan/', '/cars/catalog/chery/', '/cars/catalog/cheryexeed/', '/cars/catalog/chevrolet/', '/cars/catalog/citroen/', '/cars/catalog/dongfeng/', '/cars/catalog/faw/', '/cars/catalog/ferrari/', '/cars/catalog/fiat/', '/cars/catalog/geely/', '/cars/catalog/genesis/', '/cars/catalog/haval/', '/cars/catalog/honda/', '/cars/catalog/hyundai/', '/cars/catalog/infiniti/', '/cars/catalog/isuzu/', '/cars/catalog/jac/', '/cars/catalog/jaguar/', '/cars/catalog/jeep/', '/cars/catalog/kia/', '/cars/catalog/lada/', '/cars/catalog/lamborghini/', '/cars/catalog/land_rover/', '/cars/catalog/lexus/', '/cars/catalog/lifan/', '/cars/catalog/lotus/', '/cars/catalog/maserati/', '/cars/catalog/mazda/', '/cars/catalog/mercedes-benz/', '/cars/catalog/mini/', '/cars/catalog/mitsubishi/', '/cars/catalog/nissan/', '/cars/cata

In [ ]:
from requests.exceptions import ChunkedEncodingError, HTTPError
import time

def parse_models(brand_url, retries=3, timeout=10):
    for _ in range(retries):
        try:
            response = requests.get(brand_url, timeout=timeout)
            response.raise_for_status()  # Проверка на успешный статус ответа
            soup = BeautifulSoup(response.content, 'html.parser')

            model_names = []
            for a_tag in soup.find_all('a', class_='model-item__title'):
                model_name = a_tag.text.strip()
                model_link = f"https://automobili.ru{a_tag['href']}"
                model_names.append((model_name, model_link))

            return model_names
        except (HTTPError, ChunkedEncodingError, requests.exceptions.RequestException) as e:
            print(f"Ошибка: {e}. Повторная попытка...")
            time.sleep(2)  # Задержка перед повторной попыткой
    print(n)
    return []

all_models = {}
for brand_link in brand_links:
    full_url = f"https://automobili.ru{brand_link}"
    models = parse_models(full_url)
    all_models[brand_link] = models

print("Модели автомобилей по брендам:", all_models)

Модели автомобилей по брендам: {'/cars/catalog/aston_martin/': [('Vantage', 'https://automobili.ru/cars/catalog/aston_martin/vantage/'), ('DB11', 'https://automobili.ru/cars/catalog/aston_martin/db11/'), ('DBS Superleggera', 'https://automobili.ru/cars/catalog/aston_martin/dbs-superleggera/')], '/cars/catalog/audi/': [('A3', 'https://automobili.ru/cars/catalog/audi/a3/'), ('A4', 'https://automobili.ru/cars/catalog/audi/a4/'), ('A4 allroad', 'https://automobili.ru/cars/catalog/audi/a4_allroad/'), ('RS4 Avant', 'https://automobili.ru/cars/catalog/audi/rs4-avant/'), ('A5', 'https://automobili.ru/cars/catalog/audi/a5/'), ('S5', 'https://automobili.ru/cars/catalog/audi/s5/'), ('RS 5', 'https://automobili.ru/cars/catalog/audi/rs-5/'), ('A6', 'https://automobili.ru/cars/catalog/audi/a6/'), ('A7', 'https://automobili.ru/cars/catalog/audi/a7/'), ('A8', 'https://automobili.ru/cars/catalog/audi/a8/'), ('Q3', 'https://automobili.ru/cars/catalog/audi/q3/'), ('Q5', 'https://automobili.ru/cars/catalo

In [ ]:
import pandas as pd
import numpy as np

data = []
for brand, models in all_models.items():
    brand_name = brand.split('/')[-2]  # Извлечение названия марки из URL
    for model, link in models:
        data.append((f"{brand_name} {model}", link))

df_pars = pd.DataFrame(data, columns=['model', 'link'])

In [ ]:
df_pars.sort_values(by=['model', 'link'], inplace=True)
df_pars.head()

,model,link
1,aston_martin DB11,https://automobili.ru/cars/catalog/aston_marti...
2,aston_martin DBS Superleggera,https://automobili.ru/cars/catalog/aston_marti...
0,aston_martin Vantage,https://automobili.ru/cars/catalog/aston_marti...
3,audi A3,https://automobili.ru/cars/catalog/audi/a3/
4,audi A4,https://automobili.ru/cars/catalog/audi/a4/


In [ ]:
path = 'https://stepik.org/media/attachments/lesson/866758/quickstart_train.csv'

df = pd.read_csv(path)
df.head(3)

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,109.99,another_bug,4.737759,1.214131e+07,0.1,180.855726,0.023174,174,170
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,34.48,electro_bug,4.480517,1.803909e+07,0.0,187.862734,12.306011,174,174
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,34.93,gear_stick,4.768391,1.588366e+07,0.1,102.382857,2.513319,174,173


In [ ]:
import re
# Функция для удаления кириллицы из строки
def remove_cyrillic(text):
    return re.sub(r'[а-яА-Я]', '', text)

df_pars['model'] = df_pars['model'].apply(remove_cyrillic)
# Заменяем 'VW' на 'Volkswagen' и приводим к нижнему регистру
df['model'] = df['model'].str.replace('VW', 'Volkswagen')
df_pars['model'] = df_pars['model'].str.lower()
df['model'] = df['model'].str.lower()

In [ ]:
!pip install rapidfuzz -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 32.8 MB/s eta 0:00:00


In [ ]:
# def split_model(s):
#     parts = s.rsplit(' ', 1)
#     if len(parts) == 2:
#         return parts[0], parts[1]
#     return parts[0], ''

# def find_closest_match(row, choices, threshold=85):
#     row_base, row_suffix = split_model(row['model'])
#     base_choices = [split_model(choice)[0] for choice in choices]
#     print('row_base, row_suffix:', row_base, '|', row_suffix)
#     print('base_choices:', base_choices)

#     # Поиск наиболее близкого совпадения для основной части
#     result = process.extractOne(row_base, base_choices, scorer=fuzz.token_sort_ratio)
#     match_base, score = result[:2]
#     print('match_base, score:', match_base, '|', score)

#     if score >= threshold:
#         # Поиск оригинальной строки, соответствующей найденному совпадению
#         match = next(choice for choice in choices if split_model(choice)[0] == match_base)
#         _, match_suffix = split_model(match)
#         print('match, match_suffix:', match, '|', match_suffix)
#         print()

#         # Проверка строгого совпадения окончания
#         if row_suffix == match_suffix:
#             return match
#         else:
#             return np.nan
#     return np.nan

# Применение функции к DataFrame
#df_pars['closest_match'] = df_pars.apply(find_closest_match, axis=1, choices=df['model'])

#### В логике матчинга я исходил из того, что в написании модели допустимы различия, но в индексе справа от модели каждый символ несет смысл (MERCEDES-BENZ GLC и GLB и GLE - это все разные машины), поэтому задал алгоритм поиска 100% соответсвия в индексе

In [ ]:
def split_model(s):
    parts = s.rsplit(' ', 1)
    if len(parts) == 2:
        return parts[0], parts[1]
    return parts[0], ''

def find_closest_match(row, choices, threshold=85):
  row_base, row_suffix = split_model(row)
  print('row_base, row_suffix:', row_base, '|', row_suffix)
  for car_model in choices:
    base_choices, base_suffix = split_model(car_model)
    print('base_choices, base_suffix:', base_choices, '|', base_suffix)

    # Поиск наиболее близкого совпадения для основной части
    result = process.extractOne(row_base, [base_choices], scorer=fuzz.token_sort_ratio)
    match_base, score = result[:2]
    print('match_base, score:', match_base, '|', score)
    print()

    if score >= threshold:
        # Проверка строгого совпадения окончания
        if row_suffix == base_suffix:
            return car_model
  return np.nan

df_pars['closest_match'] = df_pars['model'].apply(find_closest_match, choices=df['model'])


Выходные данные были обрезаны до нескольких последних строк (5000).
match_base, score: nissan | 0.0

base_choices, base_suffix: audi | q3
match_base, score: audi | 0.0

base_choices, base_suffix: kia | sportage
match_base, score: kia | 0.0

base_choices, base_suffix: kia rio | x
match_base, score: kia rio | 11.764705882352944

base_choices, base_suffix: skoda | rapid
match_base, score: skoda | 13.33333333333333

base_choices, base_suffix: volkswagen polo | vi
match_base, score: volkswagen polo | 31.999999999999996

base_choices, base_suffix: kia | rio
match_base, score: kia | 0.0

base_choices, base_suffix: kia | rio
match_base, score: kia | 0.0

base_choices, base_suffix: volkswagen | tiguan
match_base, score: volkswagen | 30.000000000000004

base_choices, base_suffix: skoda | rapid
match_base, score: skoda | 13.33333333333333

base_choices, base_suffix: audi | a3
match_base, score: audi | 0.0

base_choices, base_suffix: smart | forfour
match_base, score: smart | 0.0

base_choices, ba

In [ ]:
df_pars['closest_match'].unique()

array([nan, 'audi a3', 'audi a4', 'audi q3', 'fiat 500',
       'hyundai solaris', 'kia rio', 'kia rio x', 'kia sportage',
       'mercedes-benz glc', 'nissan qashqai', 'renault kaptur',
       'renault sandero', 'skoda rapid', 'smart forfour', 'smart fortwo',
       'volkswagen polo', 'volkswagen tiguan'], dtype=object)

In [ ]:
df['model'].unique()

array(['kia rio x-line', 'volkswagen polo vi', 'renault sandero',
       'mercedes-benz glc', 'skoda rapid', 'nissan qashqai',
       'tesla model 3', 'kia sportage', 'smart forfour',
       'volkswagen polo', 'kia rio', 'smart coupe', 'volkswagen tiguan',
       'smart fortwo', 'kia rio x', 'mercedes-benz e200',
       'renault kaptur', 'mini cooper', 'hyundai solaris',
       'volkswagen id.4 ', 'fiat 500', 'bmw 320i', 'audi a4', 'audi q3',
       'audi a3', 'mini cooperse'], dtype=object)

#### Некоторые записи данных модели в исходном датасете некорректны: mercedes-benz e200, 'volkswagen polo vi, bmw 320i, volkswagen id.4 - смесь модели и модификации. А tesla model 3 вообще нет в каталоге

> Добавить блок с цитатой



In [ ]:
df_pars_cl = df_pars.dropna()

In [ ]:
from urllib.parse import urljoin

# Функция для парсинга информации о модели
def parse_model_info(model_url):
    try:
        response = requests.get(model_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Извлечение информации
        year = soup.find('li', class_='disabled').text.strip().split(' ')[-1]

        found = False
        mod = None
        td_tags = soup.find_all('td')
        for td_tag in td_tags:
          a_tag = td_tag.find('a', href=True)
          if a_tag:
            href = a_tag['href']
            full_href = urljoin(response.url, href)
            if full_href.startswith(model_url) and not found:
              mod = a_tag.text.strip()
              found = True

        price = int(soup.find('span', class_='currency').text.replace(' ', '').replace('₽', ''))
        engine = soup.find('td', class_='td_engine').text.strip()
        power = int(soup.find('td', class_='td_engine').find_next_sibling('td').text.strip().split()[0])
        box = soup.find('td', class_='td_box').text.strip()
        trans = soup.find('td', class_='td_trans').text.strip()
        body = soup.find('td', class_='td_body').text.strip()

        return year, mod, price, engine, power, box, trans, body
    except Exception as e:
        print(f"Ошибка при парсинге {model_url}: {e}")
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

# Применение функции к каждой модели
df_pars_cl[['year', 'mod', 'price', 'engine', 'power', 'box', 'trans', 'body']] = df_pars_cl['link'].apply(lambda x: parse_model_info(x) if pd.notna(x) else (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan)).apply(pd.Series)

<ipython-input-51-9d5a648d3b2a>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pars_cl[['year', 'mod', 'price', 'engine', 'power', 'box', 'trans', 'body']] = df_pars_cl['link'].apply(lambda x: parse_model_info(x) if pd.notna(x) else (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan)).apply(pd.Series)
<ipython-input-51-9d5a648d3b2a>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pars_cl[['year', 'mod', 'price', 'engine', 'power', 'box', 'trans', 'body']] = df_pars_cl['link'].apply(l

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq
0,y13744087j,kia rio x-line,economy,petrol,3.78,2015,76163,2021,109.99,another_bug,4.737759,1.214131e+07,0.1,180.855726,0.023174,174,170
1,O41613818T,volkswagen polo vi,economy,petrol,3.90,2015,78218,2021,34.48,electro_bug,4.480517,1.803909e+07,0.0,187.862734,12.306011,174,174
2,d-2109686j,renault sandero,standart,petrol,6.30,2012,23340,2017,34.93,gear_stick,4.768391,1.588366e+07,0.1,102.382857,2.513319,174,173


In [ ]:
df_pars_cl['year'] = df_pars_cl['year'].astype(int)

<ipython-input-63-d97eade8a98c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pars_cl['year'] = df_pars_cl['year'].astype(int)


In [ ]:
df_pars_cl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17 entries, 3 to 302
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   model          17 non-null     object
 1   link           17 non-null     object
 2   closest_match  17 non-null     object
 3   year           17 non-null     int64 
 4   mod            17 non-null     object
 5   price          17 non-null     int64 
 6   engine         17 non-null     object
 7   power          17 non-null     int64 
 8   box            17 non-null     object
 9   trans          17 non-null     object
 10  body           17 non-null     object
dtypes: int64(3), object(8)
memory usage: 1.6+ KB


In [ ]:
df_update = df.merge(df_pars_cl, how='left', on='model')

In [ ]:
df_update = df_update.drop(columns='closest_match')
df_update.head()

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,...,user_uniq,link,year,mod,price,engine,power,box,trans,body
0,y13744087j,kia rio x-line,economy,petrol,3.78,2015,76163,2021,109.99,another_bug,...,170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,O41613818T,volkswagen polo vi,economy,petrol,3.90,2015,78218,2021,34.48,electro_bug,...,174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,d-2109686j,renault sandero,standart,petrol,6.30,2012,23340,2017,34.93,gear_stick,...,173,https://automobili.ru/cars/catalog/renault/san...,2018.0,1.6 5MT Access,697000.0,Бензин,82.0,Механическая,Передний,Хэтчбек
3,u29695600e,mercedes-benz glc,business,petrol,4.04,2011,1263,2020,32.22,engine_fuel,...,170,https://automobili.ru/cars/catalog/mercedes-be...,2019.0,2.0 9AT Premium,4690000.0,Бензин,197.0,Классический автомат,Полный,Внедорожник
4,N-8915870N,renault sandero,standart,petrol,4.70,2012,26428,2017,27.51,engine_fuel,...,171,https://automobili.ru/cars/catalog/renault/san...,2018.0,1.6 5MT Access,697000.0,Бензин,82.0,Механическая,Передний,Хэтчбек


In [ ]:
df_update.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   car_id                    2337 non-null   object 
 1   model                     2337 non-null   object 
 2   car_type                  2337 non-null   object 
 3   fuel_type                 2337 non-null   object 
 4   car_rating                2337 non-null   float64
 5   year_to_start             2337 non-null   int64  
 6   riders                    2337 non-null   int64  
 7   year_to_work              2337 non-null   int64  
 8   target_reg                2337 non-null   float64
 9   target_class              2337 non-null   object 
 10  mean_rating               2337 non-null   float64
 11  distance_sum              2337 non-null   float64
 12  rating_min                2337 non-null   float64
 13  speed_max                 2337 non-null   float64
 14  user_rid

In [ ]:
df_update.to_csv('cars_train__updated.csv', index=False)